In [77]:
import cv2
import os
import numpy as np

In [74]:

def split_patches(img_path,slice_size,output_path):
    image = cv2.imread(img_path)
    file_name=os.path.basename(img_path)
    height, width, _ = image.shape
    slice_size = slice_size

    save_path=os.path.join(output_path,os.path.basename(img_path)[:-4])

    if os.path.isdir(save_path)==0:
        os.mkdir(save_path)
        print(f'create {save_path} directory')

    count = 0
    for start_y in range(0, height, slice_size):
        for start_x in range(0, width, slice_size):
            end_y = min(start_y + slice_size, height)
            end_x = min(start_x + slice_size, width)
            
            sliced_image = image[start_y:end_y, start_x:end_x]
            save_file_path = os.path.join(save_path, f"{file_name[:-4]}_{count:02}.png")
            cv2.imwrite(save_file_path, sliced_image)

            print(f"Saved slice {count}: {save_file_path}")
            count += 1


In [76]:
#for file in good_mask_file:
img_path=r"H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask"

image_list=os.listdir(img_path)
image_list = [file for file in image_list if file.endswith(".png")]
image_list=list(map(lambda x: os.path.join(img_path,x),image_list))

output_dir=r"H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask_split"

slice_size=512
for i in image_list:
    split_patches(i,slice_size,output_dir)

Saved slice 0: H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask_split\20241227_112114_1\20241227_112114_1_00.png
Saved slice 1: H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask_split\20241227_112114_1\20241227_112114_1_01.png
Saved slice 2: H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask_split\20241227_112114_1\20241227_112114_1_02.png
Saved slice 3: H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask_split\20241227_112114_1\20241227_112114_1_03.png
Saved slice 4: H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask_split\20241227_112114_1\20241227_112114_1_04.png
Saved slice 5: H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\JST_offer_mask_split\20241227_112114_1\20241227_11

In [90]:
def merge_patches_to_image(slice_folder,patch_size,num_patches_x,num_patches_y,save_path):
    """
    slice_folder = ['./img1.png','./img1.png','./img1.png','./img1.png',...]
    patch_size = (512, 512, 3)  # 개별 패치의 크기
    num_patches_x = 8  # 가로 패치 개수
    num_patches_y = 6  # 세로 패치 개수
    """
    full_image_size = (num_patches_y * patch_size[0], num_patches_x * patch_size[1], patch_size[2])

    # 빈 이미지 생성
    full_image = np.zeros(full_image_size, dtype=np.uint8)

    patches=[cv2.imread(i) for i in slice_folder]
  
    # 패치 합치기
    count = 0
    for y in range(num_patches_y):
        for x in range(num_patches_x):
            start_y = y * patch_size[0]
            start_x = x * patch_size[1]
            full_image[start_y:start_y + patch_size[0], start_x:start_x + patch_size[1], :] = patches[count]
            count += 1
            
    cv2.imwrite(save_path, full_image)
    print(f'save merge image in {save_path}')
    return 0



In [103]:
patch_size = (512, 512, 3)
num_patches_x = 8
num_patches_y = 6
path=r'H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\result\JST_offer\20241227_112114_1'
save_path=os.path.join(path,'20241227_111923_1_merge.png')
slice_folder=sorted(os.listdir(os.path.join(path+'/blend')))
slice_folder=list(map(lambda x: os.path.join(os.path.join(path,'blend'),x),slice_folder))
#print(slice_folder)
# print(save_path)
merge_patches_to_image(slice_folder,patch_size,num_patches_x,num_patches_y,save_path)

save merge image in H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\result\JST_offer\20241227_112114_1\20241227_111923_1_merge.png


0

In [ ]:
slice_folder=r'H:\AI-Data\New_Rule\taegwang\fitting_valve\241023 BMT_gRAND vISION_naming\outside_anomaly\result\ttest\blend'
slice_files = sorted(os.listdir(slice_folder))
number_int=list(map(lambda x: x.split('.')[0],slice_files))
number_int=list(map(lambda x: x.split('_')[-1],number_int))
number_int=list(map(lambda x: int(x),number_int[:-1]))

pair=[]
for index,file in zip(number_int,slice_files):
    pair.append((index,file))
pair=sorted(pair)
pair

In [2]:
def mask_patch_save_img(image_folder, output_path):
    image_path=os.listdir(image_folder)

    for file in image_path:

        image = cv2.imread(os.path.join(image_folder,file))
        height, width, _ = image.shape
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        _, binary = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            (x, y), r = cv2.minEnclosingCircle(largest_contour)

        if r >500: #큰원
            slice_size = 512
            count = 0
            for start_y in range(0, height, slice_size):
                for start_x in range(0, width, slice_size):

                    end_y = min(start_y + slice_size, height)
                    end_x = min(start_x + slice_size, width)
                    
                    sliced_image = image[start_y:end_y, start_x:end_x]
                    output_file = os.path.join(output_path, f"{file[:-4]}_{count}.png")
                    
                    if count in [1,2,3,4,5,6,9,14,16,17,22,23,24,25,30,31,33,38,41,42,43,44,45,46]:
                        cv2.imwrite(output_file, sliced_image)
                        print(f"Saved slice {count}: {output_file}")
                 
                    count += 1


        else : #작은원
            slice_size = 512
            count = 0
            for start_y in range(0, height, slice_size):
                for start_x in range(0, width, slice_size):
            
                    end_y = min(start_y + slice_size, height)
                    end_x = min(start_x + slice_size, width)
                    
                    sliced_image = image[start_y:end_y, start_x:end_x]
                    output_file = os.path.join(output_path, f"{file[:-4]}_{count}.png")
                    
                    if count in [10,11,12,13,18,21,26,29,34,35,36,37]:
                        cv2.imwrite(output_file, sliced_image)
                        print(f"Saved slice {count}: {output_file}")
                  

                    count += 1
        break
    return 0